In [1]:
import tensorflow as tf

In [2]:
import os
os.chdir("../")

In [5]:
ls

 Volume in drive E is New Volume
 Volume Serial Number is D2C1-552A

 Directory of e:\STUDIES\data_science\ML\Deep_Learning\Deep_CNN_classifier

21-Sep-22  03:56 PM    <DIR>          .
27-Sep-22  12:26 PM    <DIR>          ..
21-Sep-22  03:10 PM    <DIR>          .dvc
21-Sep-22  02:44 PM               142 .dvcignore
12-Sep-22  12:26 PM    <DIR>          .github
21-Sep-22  02:10 PM             2,503 .gitignore
18-Sep-22  10:17 PM    <DIR>          .mypy_cache
22-Sep-22  10:15 PM    <DIR>          .pytest_cache
19-Sep-22  01:48 AM    <DIR>          .tox
19-Sep-22  12:31 AM    <DIR>          .vscode
25-Sep-22  02:37 AM    <DIR>          artifacts
12-Sep-22  12:26 PM    <DIR>          configs
17-Sep-22  04:33 PM    <DIR>          docs
25-Sep-22  02:37 AM             1,185 dvc.lock
25-Sep-22  02:32 AM             1,460 dvc.yaml
18-Sep-22  04:36 PM    <DIR>          env
12-Sep-22  05:00 PM               300 init_setup.sh
12-Sep-22  04:04 AM             1,097 LICENSE
13-Sep-22  09:20 PM    <D

In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    params_image_size: list
    params_batch_size: int

In [10]:
from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PetImages",
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [11]:
import tensorflow as tf
from pathlib import Path

class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)


    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [12]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
    
except Exception as e:
   raise e

[2022-09-27 20:05:41,603: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-09-27 20:05:41,980: INFO: common]: yaml file: params.yaml loaded successfully
[2022-09-27 20:05:42,008: INFO: common]: created directory at: artifacts
Found 7498 images belonging to 2 classes.


: 

: 